# Context-sensitive Spelling Correction

### Student: Ivan Golov
### Email: i.golov@innopolis.university
### Group: AI-01

# Norvig's solution evaluation

I started working on the Assignment by evaluating the Norvig's solution and hightlighting the main drawbacks of it.

### Obtain the train language corpus using nltk.corpus import gutenberg, reuters, brown

In [1]:
import nltk
nltk.download('brown')
nltk.download('reuters')
nltk.download('gutenberg')
from nltk.corpus import gutenberg, reuters, brown
from tqdm import tqdm

# Function to generate large corpus text
def generate_large_corpus():
    large_corpus_text = ""

    # List of all Gutenberg file IDs
    file_ids = gutenberg.fileids()

    # Generate the large corpus by combining all texts
    large_corpus_text = "\n".join(gutenberg.raw(file_id) for file_id in file_ids)

    # Add Reuters and Brown corpora to the large corpus
    reuters_text = " ".join(reuters.words())
    brown_text = " ".join(brown.words())

    large_corpus_text += f"\n{reuters_text}\n{brown_text}"

    return large_corpus_text

# Save the large corpus to a text file
def save_large_corpus(file_path, corpus_text):
    with open(file_path, "w") as file:
        file.write(corpus_text)

# Generate and save the large corpus
large_corpus_text = generate_large_corpus()
save_large_corpus("data/train/language_corpus.txt", large_corpus_text)
print("Large corpus generated and saved to 'data/train/language_corpus.txt'")

[nltk_data] Downloading package brown to /Users/ivangolov/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     /Users/ivangolov/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/ivangolov/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Large corpus generated and saved to 'data/train/language_corpus.txt'


### Implement the Norvig model

In [2]:
import re
from collections import Counter

In [3]:
class NorvigSpellingCorrector_v1:
    def __init__(self, corpus):
        self.WORDS = Counter(self.words(corpus))
        self.N = sum(self.WORDS.values())

    def words(self, text):
        return re.findall(r'\w+', text.lower())

    def P(self, word):
        "Probability of `word`."
        return self.WORDS[word] / self.N

    def correction(self, word):
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word):
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

### Train the model

In [4]:
model = NorvigSpellingCorrector_v1(large_corpus_text)

### Test set preparation

In [5]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import re

In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ivangolov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
test_data = ""
with open('data/test/test.txt') as f:
    test_data = f.read()

In [11]:
# Tokenize the document into sentences
sentences = sent_tokenize(test_data.lower())

In [18]:
import re

# Initialize lists to store the data for the DataFrame
original_sentences = []
misspelled_sentences = []
correct_words = []
misspelled_words = []

# Define a regex pattern to find the <ERR> tags
pattern = re.compile(r'<err targ=(.*?)>(.*?)</err>')

# Define a preprocessing function
def preprocess_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove punctuation
    sentence = re.sub(r'[^\w\s]', '', sentence)
    # Remove extra whitespaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

# Process each sentence
for sentence in sentences:
    matches = pattern.findall(sentence)
    if matches:
        original_sentence = sentence
        misspelled_sentence = sentence
        correct_word_list = []
        misspelled_word_list = []
        
        for match in matches:
            correct_word = match[0]
            misspelled_word = match[1]
            misspelled_word_mew = match[1].replace(' ', '')
            correct_word_list.append(correct_word)
            misspelled_word_list.append(misspelled_word_mew)
            misspelled_sentence = misspelled_sentence.replace(f'<err targ={correct_word}>{misspelled_word}</err>', misspelled_word_mew)
            original_sentence = original_sentence.replace(f'<err targ={correct_word}>{misspelled_word}</err>', correct_word)
        
        # Apply preprocessing
        original_sentence = preprocess_sentence(original_sentence)
        misspelled_sentence = preprocess_sentence(misspelled_sentence)
        
        original_sentences.append(original_sentence)
        misspelled_sentences.append(misspelled_sentence)
        correct_words.append(correct_word_list)
        misspelled_words.append(misspelled_word_list)

In [19]:
# Construct the pandas DataFrame
df = pd.DataFrame({
    'Original Sentence': original_sentences,
    'Misspelled Sentence': misspelled_sentences,
    'Correct Words': correct_words,
    'Misspelled Words': misspelled_words
})

# Display the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv('data/test/test_data_processed.csv', index=False)

                                   Original Sentence  \
0  1 nigel thrush page 48 i have four in my famil...   
1                          my sister goes to tonbury   
2                          my mum goes out sometimes   
3  i go to bridgebrook i go out sometimes on tues...   
4  on thursday nights i go bellringing on saturda...   

                                 Misspelled Sentence      Correct Words  \
0  1 nigel thrush page 48 i have four in my famil...           [sister]   
1                             my siter go to tonbury     [sister, goes]   
2                          my mum goes out sometimes        [sometimes]   
3  i go to bridgebrook i go out sometimes on tues...  [sometimes, club]   
4  on thursday nights i go bellringing on saturda...      [bellringing]   

    Misspelled Words  
0            [siter]  
1        [siter, go]  
2        [sometimes]  
3  [sometimes, clob]  
4      [bellringing]  


In [20]:
df.head(10)

,Original Sentence,Misspelled Sentence,Correct Words,Misspelled Words
0,1 nigel thrush page 48 i have four in my famil...,1 nigel thrush page 48 i have four in my famil...,[sister],[siter]
1,my sister goes to tonbury,my siter go to tonbury,"[sister, goes]","[siter, go]"
2,my mum goes out sometimes,my mum goes out sometimes,[sometimes],[sometimes]
3,i go to bridgebrook i go out sometimes on tues...,i go to bridgebrook i go out sometimes on tues...,"[sometimes, club]","[sometimes, clob]"
4,on thursday nights i go bellringing on saturda...,on thursday nights i go bellringing on saturda...,[bellringing],[bellringing]
5,i go to bed at 10 o clock i watch tv at 5 o cl...,i go to bed at 10 o clock i wakh tv at 5 o clo...,[watch],[wakh]
6,the house is white it has stone up the front i...,the house is white it has stone up the frount ...,"[front, second]","[frount, sexeon]"
7,on monday i sometimes go down the farm in the ...,on monday i sometimes go down the farm in the ...,[watch],[wach]
8,we have got anglia like to watch cowboys,we have got anglia like to wach cowboys,"[watch, cowboys]","[wach, cowboys]"
9,on tuesday i get off the bus and sometimes in ...,on tuesday i get off the bus and sometimes in ...,"[sometimes, club]","[sometimes, colbe]"


### Metrics computation

In [3]:
import pandas as pd
import numpy as np
import math
from nltk.metrics import edit_distance as Levenshtein
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import ast

# Define the functions to calculate WER and CER
def calculate_wer(reference, corrected):
    # Calculate Word Error Rate (WER)
    reference_words = reference.split()
    corrected_words = corrected.split()

    S = Levenshtein(reference_words, corrected_words)
    I = max(0, len(corrected_words) - len(reference_words))
    D = max(0, len(reference_words) - len(corrected_words))

    N = max(len(reference_words), len(corrected_words))

    wer = (S + I + D) / N

    return wer

def calculate_cer(reference, corrected):
    # Calculate Character Error Rate (CER)
    S = Levenshtein(reference, corrected)
    I = max(0, len(corrected) - len(reference))
    D = max(0, len(reference) - len(corrected))

    N = max(len(reference), len(corrected))

    cer = (S + I + D) / N

    return cer

# Define the function to calculate accuracy
def calculate_accuracy(df):
    correct_predictions = 0
    total_predictions = 0

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        correct_words = ast.literal_eval(row['Correct Words'])
        corrected_words = row['Corrected Words']
        for correct_word, corrected_word in zip(correct_words, corrected_words):
            if correct_word == corrected_word:
                correct_predictions += 1
            total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

def calculate_perplexity_norvig(sentence, model):
    words = model.words(sentence)
    log_prob = 0

    for word in words:
        prob = model.P(word)
        if prob > 0:
            log_prob += np.log2(prob)  
        else:
            log_prob += np.log2(1 / model.N) 

    HC = -log_prob / len(words)  # Cross-entropy
    perpl = math.pow(2, HC)  # Perplexity

    return HC, perpl

def correct_sentence(sentence, model):
    return ' '.join([model.correction(word) for word in model.words(sentence)])

def correct_words(words, model):
    return [model.correction(word.strip()) for word in eval(words)]

def compute_stats(df, model, tag):
    WER = []
    CER = []
    accuracy = 0
    perplexities = []
    HCs = []

    if tag == "Norvig":
        
        print("Compute the corrected sentence and words")
        with ThreadPoolExecutor() as executor:
            df['Corrected Sentence'] = list(tqdm(executor.map(lambda sentence: correct_sentence(sentence, model), df['Misspelled Sentence']), total=len(df)))
            df['Corrected Words'] = list(tqdm(executor.map(lambda words: correct_words(words, model), df['Misspelled Words']), total=len(df)))
        
        print("Compute the WER, CER and Perplexity")
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            reference = row['Original Sentence']
            corrected = row['Corrected Sentence']
            wer = calculate_wer(reference, corrected)
            cer = calculate_cer(reference, corrected)
            WER.append(wer)
            CER.append(cer)
            HC, perplexity = calculate_perplexity_norvig(corrected, model)
            perplexities.append(perplexity)
            HCs.append(HC)
            
        
        print("Compute the accuracy")
        accuracy = calculate_accuracy(df)
            
    print('Average Word Error Rate (WER):', np.mean(WER))
    print('Average Character Error Rate (CER):', np.mean(CER))
    print(f"Accuracy: {accuracy}")
    print('Average Perplexity:', np.mean(perplexities))
    print('Average Cross-Entropy:', np.mean(HCs))
    
    return df

In [4]:
import pandas as pd
test_df = pd.read_csv('data/test/test_data_processed.csv')

### Collect statistics

In [115]:
updated = compute_stats(test_df.copy(), model, "Norvig")

Compute the corrected sentence and words


100%|██████████| 666/666 [00:23<00:00, 28.19it/s] 


Compute the WER, CER and Perplexity


100%|██████████| 666/666 [00:28<00:00, 22.97it/s]


Compute the accuracy


100%|██████████| 666/666 [00:00<00:00, 10956.08it/s]

Average Word Error Rate (WER): 0.12568611368448648
Average Character Error Rate (CER): 0.06691358406401805
Accuracy: 0.2058484032320123
Average Perplexity: 5198.027378935151
Average Cross-Entropy: 10.281997326281457


In [116]:
updated.head(10)

,Original Sentence,Misspelled Sentence,Correct Words,Misspelled Words,Corrected Sentence,Corrected Words
0,1 nigel thrush page 48 i have four in my famil...,1 nigel thrush page 48 i have four in my famil...,['sister'],['siter'],1 nigel thrush page 48 i have four in my famil...,[sister]
1,my sister goes to tonbury,my siter go to tonbury,"['sister', 'goes']","['siter', 'go']",my sister go to tilbury,"[sister, go]"
2,my mum goes out sometimes,my mum goes out sometimes,['sometimes'],['sometimes'],my mum goes out sometimes,[sometimes]
3,i go to bridgebrook i go out sometimes on tues...,i go to bridgebrook i go out sometimes on tues...,"['sometimes', 'club']","['sometimes', 'clob']",i go to bridgebrook i go out sometimes on tues...,"[sometimes, club]"
4,on thursday nights i go bellringing on saturda...,on thursday nights i go bellringing on saturda...,['bellringing'],['bellringing'],on thursday nights i go bellringing on saturda...,[bellringing]
5,i go to bed at 10 o clock i watch tv at 5 o cl...,i go to bed at 10 o clock i wakh tv at 5 o clo...,['watch'],['wakh'],i go to bed at 10 o clock i wash tv at 5 o clo...,[wash]
6,the house is white it has stone up the front i...,the house is white it has stone up the frount ...,"['front', 'second']","['frount', 'sexeon']",the house is white it has stone up the front i...,"[front, sexton]"
7,on monday i sometimes go down the farm in the ...,on monday i sometimes go down the farm in the ...,['watch'],['wach'],on monday i sometimes go down the farm in the ...,[each]
8,we have got anglia like to watch cowboys,we have got anglia like to wach cowboys,"['watch', 'cowboys']","['wach', 'cowboys']",we have got anglia like to each cowboys,"[each, cowboys]"
9,on tuesday i get off the bus and sometimes in ...,on tuesday i get off the bus and sometimes in ...,"['sometimes', 'club']","['sometimes', 'colbe']",on tuesday i get off the bus and sometimes in ...,"[sometimes, cole]"


### The Norvig spelling corrector model v1
* Average Word Error Rate (WER): 0.12577995752833032
* Average Character Error Rate (CER): 0.06695512816430119
* Accuracy: 0.2058484032320123
* Average Perplexity: 5198.027378935151
* Average Cross-Entropy: 10.281997326281457

bla bla bla

# My solution

### **Improvement №1** (add advanced text predprocessing, exclude rare words and add <UNK> token):

In [5]:
class NorvigSpellingCorrector_v2:
    def __init__(self, corpus):
        self.WORDS = self.preprocess_corpus(corpus)
        self.N = sum(self.WORDS.values())
        
    def preprocess_corpus(self, corpus):
        corpus = corpus.lower()

        corpus = re.sub(r'[^a-z0-9\s]', '', corpus)

        tokens = self.words(corpus)  

        word_counts = Counter(tokens)
        
        threshold = 10
        vocab = {word for word, count in word_counts.items() if count >= threshold}

        tokens = [token if token in vocab else "<UNK>" for token in tokens]

        word_counts = Counter(tokens)
    
        return word_counts 

    def words(self, text):
        return re.findall(r'\w+', text.lower())

    def P(self, word):
        "Probability of `word`."
        return self.WORDS[word] / self.N

    def correction(self, word):
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word):
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

### Load data and train model

In [6]:
large_corpus_text = ""
with open('data/train/language_corpus.txt') as f:
    large_corpus_text = f.read()

In [120]:
model = NorvigSpellingCorrector_v2(large_corpus_text)

### Compute metrics

In [121]:
updated = compute_stats(test_df.copy(), model, "Norvig")

Compute the corrected sentence and words


100%|██████████| 666/666 [00:31<00:00, 21.31it/s]


Compute the WER, CER and Perplexity


100%|██████████| 666/666 [00:29<00:00, 22.96it/s]


Compute the accuracy


100%|██████████| 666/666 [00:00<00:00, 9620.36it/s]

Average Word Error Rate (WER): 0.12965260361783013
Average Character Error Rate (CER): 0.06938579384714089
Accuracy: 0.22008464794151597
Average Perplexity: 4751.986770267538
Average Cross-Entropy: 10.065830773255565


In [122]:
updated.head(10)

,Original Sentence,Misspelled Sentence,Correct Words,Misspelled Words,Corrected Sentence,Corrected Words
0,1 nigel thrush page 48 i have four in my famil...,1 nigel thrush page 48 i have four in my famil...,['sister'],['siter'],1 nigel thrust page 48 i have four in my famil...,[sister]
1,my sister goes to tonbury,my siter go to tonbury,"['sister', 'goes']","['siter', 'go']",my sister go to tonbury,"[sister, go]"
2,my mum goes out sometimes,my mum goes out sometimes,['sometimes'],['sometimes'],my sum goes out sometimes,[sometimes]
3,i go to bridgebrook i go out sometimes on tues...,i go to bridgebrook i go out sometimes on tues...,"['sometimes', 'club']","['sometimes', 'clob']",i go to bridgebrook i go out sometimes on tues...,"[sometimes, club]"
4,on thursday nights i go bellringing on saturda...,on thursday nights i go bellringing on saturda...,['bellringing'],['bellringing'],on thursday nights i go bellringing on saturda...,[bellringing]
5,i go to bed at 10 o clock i watch tv at 5 o cl...,i go to bed at 10 o clock i wakh tv at 5 o clo...,['watch'],['wakh'],i go to bed at 10 o clock i wash tv at 5 o clo...,[wash]
6,the house is white it has stone up the front i...,the house is white it has stone up the frount ...,"['front', 'second']","['frount', 'sexeon']",the house is white it has stone up the front i...,"[front, seen]"
7,on monday i sometimes go down the farm in the ...,on monday i sometimes go down the farm in the ...,['watch'],['wach'],on monday i sometimes go down the farm in the ...,[each]
8,we have got anglia like to watch cowboys,we have got anglia like to wach cowboys,"['watch', 'cowboys']","['wach', 'cowboys']",we have got angle like to each cowboy,"[each, cowboy]"
9,on tuesday i get off the bus and sometimes in ...,on tuesday i get off the bus and sometimes in ...,"['sometimes', 'club']","['sometimes', 'colbe']",on tuesday i get off the bus and sometimes in ...,"[sometimes, cole]"


### The Norvig spelling corrector model v2
* Average Word Error Rate (WER): 0.12965260361783013
* Average Character Error Rate (CER): 0.06938579384714089
* Accuracy: 0.22008464794151597
* Average Perplexity: 4751.986770267538
* Average Cross-Entropy: 10.065830773255565

bla bla bla

### **Improvement №2** (Add the notion of context using N-gram models):

### Load and predprocess the data

In [5]:
large_corpus_text = ""
with open('data/train/language_corpus.txt') as f:
    large_corpus_text = f.read()

In [6]:
def preprocess_corpus(corpus):
        corpus = corpus.lower()

        corpus = re.sub(r'[^a-z0-9\s]', '', corpus)

        tokens = re.findall(r'\w+', corpus) 
        
        # exclude single character words
        tokens = [token for token in tokens if len(token) > 1]

        word_counts = Counter(tokens)
        
        threshold = 10
        vocab = {word for word, count in word_counts.items() if count >= threshold}


        return tokens, vocab

corpus_words, vocab = preprocess_corpus(large_corpus_text)

In [7]:
def preprocess_sentences(sentences, vocab):
    preprocessed_sentences = []
    for sentence in tqdm(sentences, total=len(sentences)):
        sentence = sentence.lower()
        sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
        tokens = re.findall(r'\w+', sentence)
        preprocessed_sentences.append(tokens)
    return preprocessed_sentences

In [8]:
corpus_sentences = nltk.sent_tokenize(large_corpus_text)

In [9]:
corpus_sentences_predprocessed = preprocess_sentences(corpus_sentences, vocab)

100%|██████████| 248174/248174 [00:03<00:00, 78947.85it/s] 


### Compute the N-gram stats

In [10]:
from collections import defaultdict
from tqdm import tqdm

def build_ngram_counts(words, max_order):
    """
    Build n-gram counts for orders 1 through max_order.
    For unigrams, keys are one-element tuples.
    """
    ngram_counts = {i: defaultdict(int) for i in range(1, max_order + 1)}

    for i in tqdm(range(len(words)), desc="Building n-gram counts"):
        for order in range(1, max_order + 1):
            if i + order <= len(words):
                gram = tuple(words[i:i + order])
                ngram_counts[order][gram] += 1
    
    return ngram_counts

ngram_counts = build_ngram_counts(corpus_words, 3)

Building n-gram counts: 100%|██████████| 4427046/4427046 [00:14<00:00, 314978.51it/s]


### Unigram probability model

In [11]:
def compute_unigram_probability(candidate, ngram_counts, alpha, total_words, V):
    """
    Computes smoothed unigram probability:
      P(candidate) = (C(candidate) + alpha) / (total_words + alpha * |V|)
    """
    unigram_count = ngram_counts[1].get((candidate,), 0)
    V_size = len(V)
    
    unigram_probability = (unigram_count + alpha) / (total_words + alpha * V_size)
    
    return unigram_probability

### Bigram probability model

In [12]:
# Compute the bigram probabilities
def compute_bigram_probabilities(w1, w2, ngram_counts, alpha):
    """
    Computes smoothed bigram probability:
      P(w2|w1) = (C(w1, w2) + alpha) / (C(w1) + alpha * |V|)
    """
    bigram_count = ngram_counts[2][(w1, w2)]
    unigram_count = ngram_counts[1][(w1,)]
    
    V = len(ngram_counts[1])
    
    bigram_probability = (bigram_count + alpha) / (unigram_count + alpha * V)
    
    return bigram_probability

In [13]:
def compute_perplexity_bigram(sentences, ngram_counts, alpha):
    """
    Compute the perplexity of the validation set using the bigram model.
    """
    total_log_prob = 0
    total_words = 0

    for sentence in tqdm(sentences, desc="Computing Perplexity"):
        for i in range(1, len(sentence)):
            w1 = sentence[i - 1]
            w2 = sentence[i]
            bigram_prob = compute_bigram_probabilities(w1, w2, ngram_counts, alpha)
            log_prob = np.log2(bigram_prob)
            total_log_prob += log_prob
            total_words += 1

    HC = -total_log_prob / total_words
    perplexity = math.pow(2, HC)

    return HC, perplexity

In [45]:
def compute_perplexity_bigram_avg(sentences, ngram_counts, alpha):
    """
    Compute the average perplexity of the validation set using the bigram model.
    """
    total_perplexity = 0
    num_sentences = len(sentences)

    for sentence in tqdm(sentences, desc="Computing Perplexity"):
        total_log_prob = 0
        total_words = 0
        for i in range(1, len(sentence)):
            w1 = sentence[i - 1]
            w2 = sentence[i]
            bigram_prob = compute_bigram_probabilities(w1, w2, ngram_counts, alpha)
            log_prob = np.log2(bigram_prob)
            total_log_prob += log_prob
            total_words += 1

        HC = -total_log_prob / total_words
        sentence_perplexity = math.pow(2, HC)
        total_perplexity += sentence_perplexity

    average_perplexity = total_perplexity / num_sentences

    return HC, average_perplexity

### Trigram probability model

In [14]:
# Compute the trigram probabilities
def compute_trigram_probabilities(w1, w2, w3, ngram_counts, alpha):
    """
    Computes smoothed trigram probability:
      P(w3|w1, w2) = (C(w1, w2, w3) + alpha) / (C(w1, w2) + alpha * |V|)
    """
    trigram_count = ngram_counts[3][(w1, w2, w3)]
    bigram_count = ngram_counts[2][(w1, w2)]
    
    V = len(ngram_counts[1])
    
    trigram_probability = (trigram_count + alpha) / (bigram_count + alpha * V)
    
    return trigram_probability

In [15]:
def compute_perplexity_trigram(sentences, ngram_counts, alpha):
    """
    Compute the perplexity of the validation set using the trigram model.
    """
    total_log_prob = 0
    total_words = 0

    for sentence in tqdm(sentences, desc="Computing Perplexity"):
        for i in range(2, len(sentence)):
            w1 = sentence[i - 2]
            w2 = sentence[i - 1]
            w3 = sentence[i]
            trigram_prob = compute_trigram_probabilities(w1, w2, w3, ngram_counts, alpha)
            log_prob = np.log2(trigram_prob)
            total_log_prob += log_prob
            total_words += 1

    HC = -total_log_prob / total_words
    perplexity = math.pow(2, HC)

    return HC, perplexity

### Interpolated bi-gram and tri-gram model

In [16]:
def compute_interpolated_prob(w1, w2, w3, ngram_counts, alpha, lamda):
    """
    Computes the interpolated probability:
      P(w3|w1,w2) = lam * P_trigram(w3|w1,w2) + (1 - lam) * P_bigram(w3|w2)
    where the bigram probability is computed as:
      P(w3|w2) = (C(w2, w3) + alpha) / (C(w2) + alpha * |V|)
    """
    # Trigram probability
    p_trigram = compute_trigram_probabilities(w1, w2, w3, ngram_counts, alpha)
    
    # Bigram probability
    p_bigram = compute_bigram_probabilities(w2, w3, ngram_counts, alpha)
    
    # Interpolated probability
    interpolated_prob = lamda * p_trigram + (1 - lamda) * p_bigram
    
    return interpolated_prob

In [17]:
def compute_perplexity_interpolated(sentences, ngram_counts, alpha, lamda):
    """
    Compute the perplexity of the validation set using the interpolated model.
    """
    total_log_prob = 0
    total_words = 0

    for sentence in tqdm(sentences, desc="Computing Perplexity"):
        for i in range(2, len(sentence)):
            w1 = sentence[i - 2]
            w2 = sentence[i - 1]
            w3 = sentence[i]
            interpolated_prob = compute_interpolated_prob(w1, w2, w3, ngram_counts, alpha, lamda)
            log_prob = np.log2(interpolated_prob)
            total_log_prob += log_prob
            total_words += 1

    HC = -total_log_prob / total_words
    perplexity = math.pow(2, HC)

    return HC, perplexity

In [52]:
def compute_perplexity_interpolated_avg(sentences, ngram_counts, alpha, lamda):
    """
    Compute the average perplexity of the validation set using the interpolated model.
    """
    total_perplexity = 0
    num_sentences = len(sentences)

    for sentence in tqdm(sentences, desc="Computing Perplexity"):
        total_log_prob = 0
        total_words = 0
        for i in range(2, len(sentence)):
            w1 = sentence[i - 2]
            w2 = sentence[i - 1]
            w3 = sentence[i]
            interpolated_prob = compute_interpolated_prob(w1, w2, w3, ngram_counts, alpha, lamda)
            log_prob = np.log2(interpolated_prob)
            total_log_prob += log_prob
            total_words += 1

        HC = -total_log_prob / total_words
        sentence_perplexity = math.pow(2, HC)
        total_perplexity += sentence_perplexity

    average_perplexity = total_perplexity / num_sentences

    return HC, average_perplexity

### N-gram probability model

In [18]:
def compute_ngram_probability(context, candidate, ngram_counts, alpha, V):
    """
    Computes smoothed n-gram probability:
      P(candidate|context) = (C(context, candidate) + alpha) / (C(context) + alpha * |V|)
    """
    context_length = len(context)
    ngram_count = ngram_counts.get(context_length + 1, {}).get(context + (candidate,), 0)
    context_count = ngram_counts.get(context_length, {}).get(context, 0)
    V_size = len(V)
    
    ngram_probability = (ngram_count + alpha) / (context_count + alpha * V_size)
    
    return ngram_probability

### Tuning of hyperparameters

In [ ]:
# Tune the hyperparameters for the bigram LM
validation_set = corpus_sentences[:10000]
alpha_values = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0]
lambda_values = [0.1, 0.2, 0.3, 0.5, 0.7, 0.9]

best_bigram_params=None
best_bigram_ce = float('inf')
print("Tuning hyperparameters for the bigram LM:")
for alpha in (alpha_values):
    HC, perpl = compute_perplexity_bigram(validation_set, ngram_counts, alpha)
    print(f"Alpha: {alpha}, Cross-Entropy: {HC}, Perplexity: {perpl}")
    if HC < best_bigram_ce:
        best_bigram_ce = HC
        best_bigram_params = (alpha, HC, perpl)
        
# Tune the hyperparameters for the trigram LM
best_trigram_params=None
best_trigram_ce = float('inf')
print("Tuning hyperparameters for the trigram LM:")
for alpha in (alpha_values):
    HC, perpl = compute_perplexity_trigram(validation_set, ngram_counts, alpha)
    print(f"Alpha: {alpha}, Cross-Entropy: {HC}, Perplexity: {perpl}")
    if HC < best_trigram_ce:
        best_trigram_ce = HC
        best_trigram_params = (alpha, HC, perpl)
        
# Tune the hyperparameters for the interpolated LM
best_interpolated_params=None
best_interpolated_ce = float('inf')
print("Tuning hyperparameters for the interpolated LM:")
for alpha in (alpha_values):
    for lamda in lambda_values:
        HC, perpl = compute_perplexity_interpolated(validation_set, ngram_counts, alpha, lamda)
        print(f"Alpha: {alpha}, Lambda: {lamda}, Cross-Entropy: {HC}, Perplexity: {perpl}")
        if HC < best_interpolated_ce:
            best_interpolated_ce = HC
            best_interpolated_params = (alpha, lamda, HC, perpl)


Tuning hyperparameters for the bigram LM:


Computing Perplexity: 100%|██████████| 10000/10000 [00:01<00:00, 5044.64it/s]


Alpha: 0.0001, Cross-Entropy: 15.571766524590013, Perplexity: 48704.46861441161


Computing Perplexity: 100%|██████████| 10000/10000 [00:01<00:00, 5151.46it/s]


Alpha: 0.001, Cross-Entropy: 14.330228241264058, Perplexity: 20598.16559884154


...

Alpha: 1.0, Lambda: 0.5, Cross-Entropy: 13.82730146463481, Perplexity: 14535.584492750848


Computing Perplexity: 100%|██████████| 10000/10000 [00:03<00:00, 2976.21it/s]


Alpha: 1.0, Lambda: 0.7, Cross-Entropy: 13.932713511762609, Perplexity: 15637.40513465185


Computing Perplexity: 100%|██████████| 10000/10000 [00:03<00:00, 2822.45it/s]

Alpha: 1.0, Lambda: 0.9, Cross-Entropy: 14.101701637133313, Perplexity: 17580.66031082511


In [172]:
# Print best paraneters
print("Best hyperparameters for the bigram LM:")
print(f"Alpha: {best_bigram_params[0]}, Cross-Entropy: {best_bigram_params[1]}, Perplexity: {best_bigram_params[2]}")
print("Best hyperparameters for the trigram LM:")
print(f"Alpha: {best_trigram_params[0]}, Cross-Entropy: {best_trigram_params[1]}, Perplexity: {best_trigram_params[2]}")
print("Best hyperparameters for the interpolated LM:")
print(f"Alpha: {best_interpolated_params[0]}, Lambda: {best_interpolated_params[1]}, Cross-Entropy: {best_interpolated_params[2]}, Perplexity: {best_interpolated_params[3]}")


Best hyperparameters for the bigram LM:
Alpha: 0.1, Cross-Entropy: 13.225124444839032, Perplexity: 9575.449149327107
Best hyperparameters for the trigram LM:
Alpha: 0.01, Cross-Entropy: 14.159908223480418, Perplexity: 18304.46800107179
Best hyperparameters for the interpolated LM:
Alpha: 0.001, Lambda: 0.5, Cross-Entropy: 12.865048054124225, Perplexity: 7460.455769840212


In [19]:
import json

In [ ]:
# save the best hyperparameters
best_hyperparameters = {
    "bigram": best_bigram_params,
    "trigram": best_trigram_params,
    "interpolated": best_interpolated_params
}


with open('data/train/best_hyperparameters.json', 'w') as f:
    json.dump(best_hyperparameters, f)

### Context-Sensitive Correction based on N-gram model and beam seacrh approach

In [20]:
best_hyperparameters = None
with open('data/train/best_hyperparameters.json', 'r') as f:
    best_hyperparameters = json.load(f)

In [21]:
best_bigram_alpha = best_hyperparameters['bigram'][0]
best_trigram_alpha = best_hyperparameters['trigram'][0]
best_interpolated_alpha = best_hyperparameters['interpolated'][0]
best_interpolated_lambda = best_hyperparameters['interpolated'][1]

In [22]:
from functools import lru_cache
@lru_cache(maxsize=None)
def get_candidates(word, V):
    def known(words, V):
        return set(w for w in words if w in V)
    
    def edits1(word):
        "All edits that are one edit away from `word`."
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def edits2(word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    
    def edits3(word):
        "All edits that are three edits away from `word`."
        return (e3 for e2 in edits1(word) for e3 in edits1(e2))
    
    return known([word], V) or known(edits1(word), V) or known(edits2(word), V) or known(edits3(word), V) or {word}

In [23]:
import math
import heapq

def corrector_with_beam_search(sentence, V, ngram_counts, beam_width=2, n=3, alpha=0.01, lamda=0.5):
    sentence_tokens = preprocess_sentences([sentence], V)[0]
    candidates_sequences = [(0.0, [])]
    total_words = len(V)
    
    for word in sentence_tokens:
        new_candidates_sequences = []
        candidate_list = get_candidates(word, V)
        
        for candidate in candidate_list:
            for score, sequence in candidates_sequences:
                context_length = min(n - 1, len(sequence))
                if context_length == 0:
                    prob = compute_unigram_probability(candidate, ngram_counts, alpha, total_words, V)
                elif context_length == 1:
                    context = tuple(sequence[-1:])
                    prob = compute_bigram_probabilities(context[0], candidate, ngram_counts, best_bigram_alpha)
                elif context_length == 2:
                    w1, w2 = sequence[-2], sequence[-1]
                    prob = compute_interpolated_prob(w1, w2, candidate, ngram_counts, best_interpolated_alpha, best_interpolated_lambda)
                else:
                    context = tuple(sequence[-context_length:])
                    prob = compute_ngram_probability(context, candidate, ngram_counts, alpha, V)
                
                new_score = score + math.log(prob)
                new_seq = sequence + [candidate]
                heapq.heappush(new_candidates_sequences, (new_score, new_seq))
                if len(new_candidates_sequences) > beam_width:
                    heapq.heappop(new_candidates_sequences)
        
        candidates_sequences = new_candidates_sequences
    
    best_score, best_seq = max(candidates_sequences, key=lambda x: x[0])
    return ' '.join(best_seq)

In [24]:
# Correct vocab into a frozenset for the lrucache decorator
vocabulary = frozenset(vocab)

In [25]:
# Test example for unigram
text = "Seh is ging t te perk"
corrector_with_beam_search(text, vocabulary, ngram_counts, n=1)

100%|██████████| 1/1 [00:00<00:00, 10645.44it/s]


'she is king to the per'

In [26]:
# Test example for bigram
text = "Seh is ging t te perk"
corrector_with_beam_search(text, vocabulary, ngram_counts, n=2)

100%|██████████| 1/1 [00:00<00:00, 10205.12it/s]


'she is going to the park'

In [27]:
# Test example for trigram
text = "Seh is ging t te perk"
corrector_with_beam_search(text, vocabulary, ngram_counts, n=3)

100%|██████████| 1/1 [00:00<00:00, 2605.16it/s]


'she is going to the park'

In [28]:
# Test example for N-gram model
text = "Seh is ging t te perk"
corrector_with_beam_search(text, vocabulary, ngram_counts, n=4)

100%|██████████| 1/1 [00:00<00:00, 11366.68it/s]


'she is going tx ye peru'

### Computes metrics

In [53]:
# Define the functions to calculate WER and CER
def calculate_wer(reference, corrected):
    # Calculate Word Error Rate (WER)
    reference_words = reference.split()
    corrected_words = corrected.split()

    S = Levenshtein(reference_words, corrected_words)
    I = max(0, len(corrected_words) - len(reference_words))
    D = max(0, len(reference_words) - len(corrected_words))

    N = max(len(reference_words), len(corrected_words))

    wer = (S + I + D) / N

    return wer

def calculate_cer(reference, corrected):
    # Calculate Character Error Rate (CER)
    S = Levenshtein(reference, corrected)
    I = max(0, len(corrected) - len(reference))
    D = max(0, len(reference) - len(corrected))

    N = max(len(reference), len(corrected))

    cer = (S + I + D) / N

    return cer

def compute_metrics_for_ngram_models(df, vocabulary, ngram_counts, n = 3):
    WER = []
    CER = []
    
    corrected_sentences = []
    correct_predictions = 0
    total_predictions = 0
    # Correct the sentences using the beam search corrector
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        misspelled_sentence = row['Misspelled Sentence']
        reference = row['Original Sentence']
        corrected_sentence = corrector_with_beam_search(misspelled_sentence, vocabulary, ngram_counts, n=n)
        corrected_sentences.append(corrected_sentence)
                
        # Calculate WER and CER
        wer = calculate_wer(reference, corrected_sentence)
        cer = calculate_cer(reference, corrected_sentence)
        WER.append(wer)
        CER.append(cer)
        
        # Compute the accuracy
        list_of_corrected_words = ast.literal_eval(row['Correct Words'])
        for word in list_of_corrected_words:
            if word in corrected_sentence:
                correct_predictions += 1
            total_predictions += 1
                    
    accuracy = correct_predictions / total_predictions
    
    if n == 2:
        sentences = corrected_sentences
        HC, perplexity = compute_perplexity_bigram_avg(sentences, ngram_counts, best_bigram_alpha)
        print(f"Perplexity: {perplexity}")
        print(f"Cross-Entropy: {HC}")
    if n == 3:
        sentences = corrected_sentences
        HC, perplexity = compute_perplexity_interpolated_avg(sentences, ngram_counts, best_interpolated_alpha, best_interpolated_lambda)
        print(f"Perplexity: {perplexity}")
        print(f"Cross-Entropy: {HC}")
        
        
    print('Average Word Error Rate (WER):', np.mean(WER))
    print('Average Character Error Rate (CER):', np.mean(CER))
    print(f"Accuracy: {accuracy}")
    df["Corrected Sentence"] = corrected_sentences
    return df

### Interpolated model perfomance

In [54]:
updated = compute_metrics_for_ngram_models(test_df.copy(), vocabulary, ngram_counts, n=3)

Computing Perplexity: 100%|██████████| 666/666 [00:00<00:00, 2386.06it/s]

Perplexity: 98373.00000000065
Cross-Entropy: 16.585974779349506
Average Word Error Rate (WER): 0.18588203716488597
Average Character Error Rate (CER): 0.0815851399508757
Accuracy: 0.360908041554444


In [49]:
# print examples of fixes
for i in range(10):
    print(f"Misspelled Sentence: {test_df['Misspelled Sentence'][i]}")
    print(f"Misspleed Words: {test_df['Misspelled Words'][i]}")
    print(f"Corrected: {updated['Corrected Sentence'][i]}")
    print(f"Target: {test_df['Original Sentence'][i]}")
    print("-----------------------------")

Misspelled Sentence: 1 nigel thrush page 48 i have four in my family dad mum and siter
Misspleed Words: ['siter']
Corrected: m1 nigel thrust page 48 in have four in my family dad sum and sister
Target: 1 nigel thrush page 48 i have four in my family dad mum and sister
-----------------------------
Misspelled Sentence: my siter go to tonbury
Misspleed Words: ['siter', 'go']
Corrected: my sister go to tonbury
Target: my sister goes to tonbury
-----------------------------
Misspelled Sentence: my mum goes out sometimes
Misspleed Words: ['sometimes']
Corrected: my um goes out sometimes
Target: my mum goes out sometimes
-----------------------------
Misspelled Sentence: i go to bridgebrook i go out sometimes on tuesday night i go to youth clob
Misspleed Words: ['sometimes', 'clob']
Corrected: it go to bridgebrook xi go out sometimes on tuesday night in go to youth club
Target: i go to bridgebrook i go out sometimes on tuesday night i go to youth club
-----------------------------
Misspelled

### Bigram model perfomance

In [60]:
updated = compute_metrics_for_ngram_models(test_df.copy(), vocabulary, ngram_counts, n=2)

Computing Perplexity: 100%|██████████| 666/666 [00:00<00:00, 4257.11it/s]

Perplexity: 98373.00000000065
Cross-Entropy: 16.585974779349506
Average Word Error Rate (WER): 0.1869153014145821
Average Character Error Rate (CER): 0.08206445300109326
Accuracy: 0.3528280107733744


In [61]:
# print examples of fixes
for i in range(10):
    print(f"Misspelled Sentence: {test_df['Misspelled Sentence'][i]}")
    print(f"Misspleed Words: {test_df['Misspelled Words'][i]}")
    print(f"Corrected: {updated['Corrected Sentence'][i]}")
    print(f"Target: {test_df['Original Sentence'][i]}")
    print("-----------------------------")

Misspelled Sentence: 1 nigel thrush page 48 i have four in my family dad mum and siter
Misspleed Words: ['siter']
Corrected: m1 nigel thrust page 48 in have four in my family dad sum and sister
Target: 1 nigel thrush page 48 i have four in my family dad mum and sister
-----------------------------
Misspelled Sentence: my siter go to tonbury
Misspleed Words: ['siter', 'go']
Corrected: my sister go to tonbury
Target: my sister goes to tonbury
-----------------------------
Misspelled Sentence: my mum goes out sometimes
Misspleed Words: ['sometimes']
Corrected: my um goes out sometimes
Target: my mum goes out sometimes
-----------------------------
Misspelled Sentence: i go to bridgebrook i go out sometimes on tuesday night i go to youth clob
Misspleed Words: ['sometimes', 'clob']
Corrected: it go to bridgebrook xi go out sometimes on tuesday night in go to youth club
Target: i go to bridgebrook i go out sometimes on tuesday night i go to youth club
-----------------------------
Misspelled

# Solution №2 N-gram model based on the Google Books n-gram API

In [50]:
import re
import requests
import urllib

class ContextualSpellingCorrector:
    def __init__(self, vocabulary):
        self.vocabulary = set(vocabulary)

    def run_query(self, query, start_year=2010, end_year=2019, corpus=26, smoothing=3):
        """Fetches frequency data from the Google Books Ngram API."""
        query = urllib.parse.quote(query)
        url = f'https://books.google.com/ngrams/json?content={query}&year_start={start_year}&year_end={end_year}&corpus={corpus}&smoothing={smoothing}'
        response = requests.get(url)
        
        try:
            output = response.json()
        except:
            return {}

        if not output:
            return {}

        return {entry['ngram']: sum(entry['timeseries']) / len(entry['timeseries']) for entry in output}

    def average_frequency(self, phrase):
        """Gets the average frequency of a word or n-gram phrase from Google Ngrams."""
        data = self.run_query(phrase)
        return sum(data.values()) / len(data) if data else 0

    def words(self, text):
        """Tokenizes and lowercases the input text."""
        return re.findall(r'\w+', text.lower())

    def correction_with_context(self, word, context_window):
        """Finds the best spelling correction by considering context-based n-gram probabilities."""
        candidates = self.candidates(word)
        
        # Generate n-grams with the surrounding context
        context_phrases = {candidate: self.form_context_phrases(candidate, context_window) for candidate in candidates}

        # Get frequencies for each candidate within its context
        context_frequencies = {
            candidate: sum(self.average_frequency(phrase) for phrase in phrases)
            for candidate, phrases in context_phrases.items()
        }

        return max(context_frequencies, key=context_frequencies.get)  # Return the best correction

    def form_context_phrases(self, candidate, context_window):
        """Forms bigram and trigram phrases including the candidate."""
        left_context, right_context = context_window
        phrases = []

        if left_context:
            phrases.append(f"{left_context} {candidate}")
        if right_context:
            phrases.append(f"{candidate} {right_context}")
        if left_context and right_context:
            phrases.append(f"{left_context} {candidate} {right_context}")

        return phrases

    def candidates(self, word):
        """Generates possible spelling corrections based on known words."""
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        """Filters words that exist in the vocabulary."""
        return set(w for w in words if w in self.vocabulary)

    def edits1(self, word):
        """Generates possible single-edit variations of a word."""
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """Generates possible double-edit variations of a word."""
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

# Load corpus and build vocabulary
with open("data/train/large_corpus.txt") as f:
    train_corpus = f.read()

vocabulary = set(re.findall(r'\w+', train_corpus.lower()))
vocabulary.update(["<START>", "<END>"])  # Add special tokens

# Initialize corrector with updated vocabulary
corrector = ContextualSpellingCorrector(vocabulary)

In [ ]:
# Initialize corrector
corrector = ContextualSpellingCorrector(vocabulary)

# Example usage with context-aware spelling correction
sentence = ["this", "is", "a", "speling", "error"]
corrected_list = []

for i, word in enumerate(sentence):
    left_context = sentence[i - 1] if i > 0 else "<START>"
    right_context = sentence[i + 1] if i < len(sentence) - 1 else "<END>"
    
    corrected_word = corrector.correction_with_context(word, (left_context, right_context))
    corrected_list.append(corrected_word)

corrected_sentence = " ".join(corrected_list)
print(f"Corrected sentence: {corrected_sentence}")

In [ ]:
# Process each sentence
corrected_sentences = []
corrected_words_list = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    misspelled_sentence = word_tokenize(row["Misspelled Sentence"])  # Tokenize sentence
    corrected_words = []  # Store corrected words
    
    corrected_sentence = []
    for i, word in enumerate(misspelled_sentence):
        left_context = misspelled_sentence[i - 1] if i > 0 else "<START>"
        right_context = misspelled_sentence[i + 1] if i < len(misspelled_sentence) - 1 else "<END>"

        corrected_word = corrector.correction_with_context(word, (left_context, right_context))
        corrected_sentence.append(corrected_word)
        
        # If the word was corrected, add it to the corrected words list
        if corrected_word != word:
            corrected_words.append((word, corrected_word))

    # Store results
    corrected_sentences.append(" ".join(corrected_sentence))
    corrected_words_list.append(corrected_words)
    break

# Add results to DataFrame
print(corrected_sentences)
print(corrected_words_list)